In [9]:
from fastdtw import fastdtw
import numpy as np

def dtw(x, y):
    distance, path = fastdtw(x, y)
    return distance


def paa(x, w):
    #w is the number of segments
    #first calculate the length of the time series
    n = len(x)
    #now calculate the length of each segment
    step = n/w
    #now calculate the paa
    paa = []
    for i in range(w):
        paa.append(np.mean(x[i:i+int(step)]))
        
    return paa

x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16.12,34,11,12,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
w = 16
print(len(paa(x,w)))






16


In [1]:
from elasticsearch import Elasticsearch
from elastic import es

index_settings = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 0  # Adjust as needed
    },
    'mappings': {
        'properties': {
            'dense_vector_field': {
                'type': 'dense_vector',
                'dims': 16  # Number of dimensions in the vector
            },
            # Add other fields as needed
        }
    }
}

print(es.ping())

es.indices.create(index="test2", body=index_settings)
mappings = es.indices.get_mapping(index="test2")
print(mappings)

True
{'test2': {'mappings': {'properties': {'dense_vector_field': {'type': 'dense_vector', 'dims': 16, 'index': True, 'similarity': 'cosine'}}}}}


In [62]:
import os
for filename in os.listdir('index_dataset/index_beetles/rel'):
    file = open('index_dataset/index_beetles/rel/'+filename, 'r')
    content = file.read()
    lines = content.split('\n')
    #remove last element
    lines = lines[:-1]
    
    for note in lines:
        #first take the 16th order paa of the note
        note = note.split(' ')
        
        if note == ['']:
            continue
        if len(note) < 3:
            continue
        elif len(note)>3 and len(note)<16:
            continue
        else:
            note.pop()
            note = [float(i) for i in note]
            note = paa(note, 16)
            try:
                es.index(index="test2", body={"name": filename, "paa_16": note})
            except Exception as e:
                continue
            
        
        
        
    
    

c:\Users\soumil\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\soumil\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
#show the index
query = {
           "field": "dense_vector_field",
            "query_vector": [0,1.2730118181818182, 0.3010893636363636, -0.5181818181818181, -1.053568181818182, -1.0633636363636365, -0.5374554545454546, 0.2816765363636364, 1.2399245454545456, 1.2639118181818183, 0.29585163636363637, -0.52199, -1.0546354545454546, -1.0591845454545454, -0.5086163636363636, 0.33206863636363637],
            "k": 5,
            "num_candidates": 10
        }
    
results = es.knn_search(index="test2", knn=query, source=True)
print(results['hits']['hits'])

[]


C:\Users\soumil\AppData\Local\Temp\ipykernel_22424\3529342300.py:9: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  results = es.knn_search(index="test2", knn=query, source=True)


In [13]:
import numpy as np
import feature_extraction.extract as extract

# Your input file
input_file = "user_input/query.wav"

# Extract features
absolute_pitch, relative_pitch, tempo = extract.extract(input_file, apitch=True, tempo=True, rpitch=True)

# Perform PAA on relative_pitch
paa_16 = paa(relative_pitch, 16)
print(paa_16)

query = {
           "field": "dense_vector_field",
            "query_vector": paa_16,
            "k": 5,
            "num_candidates": 10
        }

results = es.knn_search(index="test2", knn=query, source=True)

# Extract and print the results
for hit in results['hits']['hits']:
    source = hit['_source']
    print(f"Document ID: {hit['_id']}, Score: {hit['_score']}, Dense Vector: {source['dense_vector_field']}")


Predicting MIDI for user_input/query.wav...
[-2.0, -2.0, 0.0, 0.0, 1.5, 2.0, 1.5, -2.0, -2.5, 13.0, -1.5, -14.0, 1.0, 1.5, 3.5, 3.0]


C:\Users\soumil\AppData\Local\Temp\ipykernel_22424\2338202644.py:21: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  results = es.knn_search(index="test2", knn=query, source=True)
